# 컬러이미지 처리

#### 1. Pillow library 사용
  - 
  

In [1]:
from tkinter import *
from tkinter.simpledialog import *
from tkinter.filedialog import *
import math
import os
import os.path
from PIL import Image, ImageFilter, ImageEnhance, ImageOps
import time
import numpy as np

# 이미지 출력

In [2]:
def malloc(h, w, initValue=0) :    
    ImageMemory = []
    for _ in range(3):    
        retMemory=[]
        for _ in range(h):
            tmpList = []
            for _ in range(w):
                tmpList.append(initValue)
            retMemory.append(tmpList)
        ImageMemory.append(retMemory)
    
    return ImageMemory

# 파일을 메모리로 로딩하는 함수
def loadImageColor(fname) :
    global window, canvas, paper, filename, inImage, outImage, inH, inW, outH, outW
    photo = Image.open(fname) # PIL 객체
    inW = photo.width; inH = photo.height
    
    inImage = malloc(inH, inW)
    
    photoRGB = photo.convert('RGB')
    for i in range(inH):
        for k in range(inW):
            r, g, b = photoRGB.getpixel((k,i))
            inImage[R][i][k] = r
            inImage[G][i][k] = g
            inImage[B][i][k] = b

def openImageColor() :
    global window, canvas, paper, filename, inImage, outImage,inH, inW, outH, outW
    filename = askopenfilename(parent=window,
                filetypes=(("칼라 파일", "*.jpg;*.png;*.bmp;*.tif"), ("모든 파일", "*.*")))
    if filename == '' or filename == None :
        return
    loadImageColor(filename)
    equalImageColor()

    displayImageColor()

def displayImageColor() :
    global window, canvas, paper, filename, inImage, outImage, inH, inW, outH, outW
    if canvas != None : # 예전에 실행한 적이 있다.
        canvas.destroy()
    VIEW_X = outW;    VIEW_Y = outH;   step = 1

    window.geometry(str(int(VIEW_Y*1.2)) + 'x' + str(int(VIEW_X*1.2)))  # 벽
    canvas = Canvas(window, height=VIEW_Y, width=VIEW_X)
    paper = PhotoImage(height=VIEW_Y, width=VIEW_X)
    canvas.create_image((VIEW_X // 2, VIEW_Y // 2), image=paper, state='normal')

    rgbStr = '' # 전체 픽셀의 문자열을 저장
    for i in np.arange(0,outH, step) :
        tmpStr = ''
        for k in np.arange(0,outW, step) :
            i = int(i); k = int(k)
            r, g, b = outImage[0][i][k], outImage[1][i][k],outImage[2][i][k]
            tmpStr += ' #%02x%02x%02x' % (r,g,b)
        rgbStr += '{' + tmpStr + '} '
    paper.put(rgbStr)

    canvas.pack(expand=1, anchor=CENTER)
    status.configure(text='이미지 정보:' + str(outW) + 'x' + str(outH))

def saveImagePIL():
    global window, canvas, paper, filename, inImage, outImage, inH, inW, outH, outW
    if outImage == None :
        return
    saveFp = asksaveasfile(parent=window, mode='wb',
                           defaultextension='*.jpg', filetypes=(("JPG 파일", "*.jpg"), ("모든 파일", "*.*")))
    if saveFp == '' or saveFp == None:
        return
    

def  equalImageColor() :
    global window, canvas, paper, filename, inImage, outImage, inH, inW, outH, outW
    
    # outImage 메모리 확보하기 ############
    outH = inH;  outW = inW;
    outImage = []
    for _ in range(3):
        outImage.append(malloc(outH,outW))
    #######################################
    # Equal Image 알고리즘 - 출력 이미지에 입력 이미지의 RGB 값을 똑같이 넣는다.
    for RGB in range(3):
        for i in range(inH):
            for k in range(inW):
                outImage[RGB][i][k] = inImage[RGB][i][k]
    ###################################################

# CV 알고리즘

### 화소점 처리

- 이미지 밝기 조절
  - 

In [3]:
def addImageColor() :
    global window, canvas, paper, filename, inImage, outImage, inH, inW, outH, outW
    value = askinteger("밝게/어둡게", "값(-255~255)-->", minvalue=-255, maxvalue=255)
    # outImage 메모리 확보하기 ############
    outH = inH;  outW = inW;
    outImage = malloc(outH, outW)
    #######################################
    # 이미지 밝기 알고리즘
    for RGB in range(3):
        for i in range(inH):
            for k in range(inW):
                if outImage[RGB][i][k] + value > 255:
                    outImage[RGB][i][k] = 255
                elif outImage[RGB][i][k] + value < 0:
                    outImage[RGB][i][k] = 0
                else:
                    outImage[RGB][i][k] = inImage[RGB][i][k] + value
    displayImageColor()
    
def revImageColor():
    global inImage, outImage ,inH, inW, outH, outW, window, canvas, paper, filename
    # outImage 메모리 확보하기 ############
    outH = inH;  outW = inW;
    outImage = malloc(outH, outW)
    #######################################
    # Reverse Image 알고리즘     
    for RGB in range(3):
        for i in range(outH):
            for k in range(outW):
                    outImage[RGB][i][k] = 255 - inImage[RGB][i][k]
    displayImageColor()

def paraImageColor():
    global inImage, outImage ,inH, inW, outH, outW, window, canvas, paper, filename
    # outImage 메모리 확보하기 ############
    outH = inH;  outW = inW;
    outImage = malloc(outH, outW)
    #######################################
    # 파라볼라 알고리즘 Look Up TAble 사용
    LUT = [0 for _ in range(256)]
    for input in range(256):
        LUT[input] = int(255 - 255 * math.pow(input/128 -1, 2)) 
    for RGB in range(3):
        for i in range(outH):
            for k in range(outW):
                    outImage[RGB][i][k] = LUT[inImage[RGB][i][k]]
    displayImageColor()

def morphImageColor() :
    global window, canvas, paper, filename, inImage, outImage, inH, inW, outH, outW
    ## 중요! 코드. 출력영상 크기 결정 ##
    outH = inH;
    outW = inW;
    ## 추가 영상 선택
    filename2 = askopenfilename(parent=window,
                               filetypes=(("칼라 파일", "*.jpg;*.png;*.bmp;*.tif"), ("모든 파일", "*.*")))
    if filename2 == '' or filename2 == None:
        return
    inImage2 = []
    photo2 = Image.open(filename2) # PIL 객체
    inW2 = photo2.width; inH2=photo2.height
    
    ## 메모리 확보
    inImage2 = malloc(inH2, inW2)

    photoRGB2 = photo2.convert('RGB')
    for i in range(inH2) :
        for k in range(inW2) :
            r, g, b = photoRGB2.getpixel((k,i))
            inImage2[R][i][k] = r
            inImage2[G][i][k] = g
            inImage2[B][i][k] = b






### 기하처리 알고리즘

In [4]:
def  updownImageColor() :
    global window, canvas, paper, filename, inImage, outImage, inH, inW, outH, outW
    ## 중요! 코드. 출력영상 크기 결정 ##
    outH = inH;  outW = inW;
    ###### 메모리 할당 ################
    outImage = malloc(outH, outW)
    ####### 진짜 컴퓨터 비전 알고리즘 #####
    for RGB in range(3) :
        for i in range(inH) :
            for k in range(inW) :
                outImage[RGB][inH-i-1][k] = inImage[RGB][i][k]

    displayImageColor()

# 영상 축소 알고리즘
def  zoomoutImageColor() :
    global window, canvas, paper, filename, inImage, outImage, inH, inW, outH, outW
    scale = askinteger("축소", "값-->", minvalue=2, maxvalue=16)
    ## 중요! 코드. 출력영상 크기 결정 ##
    outH = inH//scale;  outW = inW//scale;
    ###### 메모리 할당 ################
    outImage = malloc(outH, outW)
    ####### 진짜 컴퓨터 비전 알고리즘 #####
    for RGB in range(3) :
        for i in range(outH) :
            for k in range(outW) :
                outImage[RGB][i][k] = inImage[RGB][i*scale][k*scale]

    displayImageColor()

# 영상 확대 알고리즘
def  zoominImageColor() :
    global window, canvas, paper, filename, inImage, outImage, inH, inW, outH, outW
    scale = askinteger("확대", "값-->", minvalue=2, maxvalue=8)
    ## 중요! 코드. 출력영상 크기 결정 ##
    outH = inH*scale;  outW = inW*scale;
    ###### 메모리 할당 ################
    outImage = malloc(outH, outW)
    ####### 진짜 컴퓨터 비전 알고리즘 #####
    for RGB in range(3) :
        for i in range(outH) :
            for k in range(outW) :
                outImage[RGB][i][k] = inImage[RGB][i//scale][k//scale]

    displayImageColor()

def moveImageColor():
    pass
def clock90ImageColor():
    pass
def rotateImageColor():
    pass

# 전역변수 선언

In [5]:
R, G, B = 0, 1, 2
inImage, outImage = [], []  # 3차원 리스트(배열)
inH, inW, outH, outW = [0] * 4
window, canvas, paper = None, None, None
filename = ""
VIEW_X, VIEW_Y = 512, 512 # 화면에 보일 크기 (출력용)


# 메인코드부




Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-2-9df806449e0e>", line 37, in openImageColor
    equalImageColor()
  File "<ipython-input-2-9df806449e0e>", line 88, in equalImageColor
    outImage[RGB][i][k] = inImage[RGB][i][k]
IndexError: list index out of range


comVisionMenu2 = Menu(mainMenu)
mainMenu.add_cascade(label="화소 (통계)", menu=comVisionMenu2)
comVisionMenu2.add_command(label="이진화 (= 흑백)", command=bwImage)
comVisionMenu2.add_command(label="입력/출력 영상 평균값", command=avgImage)
comVisionMenu2.add_command(label="축소 (평균 변환)", command=downsizeImage)
comVisionMenu2.add_command(label="확대 (양선형 보간)", command=upsizeImage2)
comVisionMenu2.add_separator()
comVisionMenu2.add_command(label="히스토그램", command=histoImage)
comVisionMenu2.add_command(label="히스토그램 (시각화 연습)", command=histoImage2)
comVisionMenu2.add_command(label="명암대비", command=stretchImage)
comVisionMenu2.add_command(label="End-In 탐색", command=endinImage)
comVisionMenu2.add_command(label="히스토그램 평활화", command=histoeqImage)

comVisionMenu3 = Menu(mainMenu)
mainMenu.add_cascade(label="기하학 처리", menu=comVisionMenu3)
comVisionMenu3.add_command(label="상하 반전", command=updownImage)
comVisionMenu3.add_command(label="이동 (상하/좌우)", command=moveImage)
comVisionMenu3.add_command(label="확대", command=upsizeImage)
comVisionMenu3.add_command(label="축소", command=downsizeImage2)
comVisionMenu3.add_command(label="오른쪽 90도 회전", command=clock90Image)
comVisionMenu3.add_command(label="회전", command=rotateImage)
comVisionMenu3.add_command(label="회전2 (중심, 역방향)", command=rotateImage2)

comVisionMenu4 = Menu(mainMenu)
mainMenu.add_cascade(label="화소 영역 처리", menu=comVisionMenu4)
comVisionMenu4.add_command(label="엠보싱 처리", command=embossImage)
comVisionMenu4.add_command(label="블러 처리", command=blurImage)
comVisionMenu4.add_command(label="샤프닝 처리", command=sharpenImage)
comVisionMenu4.add_command(label="가우시안 필터링", command=gaussImage)
comVisionMenu4.add_command(label="고주파 필터 샤프닝", command=hpfsharpenImage)
comVisionMenu4.add_command(label="저주파 필터 샤프닝", command=lpfsharpenImage)
comVisionMenu4.add_command(label="경계선 검출", command=edgeImage)

comVisionMenu5 = Menu(mainMenu)
mainMenu.add_cascade(label="기타 입출력", menu=comVisionMenu5)
comVisionMenu5.add_command(label="MySQL에서 불러오기", command=loadMysql)
comVisionMenu5.add_command(label="MySQL로 저장하기", command=saveMysql)
comVisionMenu2.add_separator()
comVisionMenu5.add_command(label="CSV 열기", command=openCsv)
comVisionMenu5.add_command(label="CSV 저장", command=saveCsv)
comVisionMenu2.add_separator()
comVisionMenu5.add_command(label="엑셀 열기", command=openExcel)
comVisionMenu5.add_command(label="엑셀 저장", command=saveExcel)
comVisionMenu5.add_command(label="엑셀 아트로 저장", command=saveExcelArt)
